In [1]:
### Parameters
#fold=1
#dataset='nursery' #'NHPA'
#d = 2# depth
#epsilon=1


In [2]:
### FAIRNESS SETS and PARAMS

#sensitive_features=[5]#[38,39,40,41,42]

## Create Cython Modules

In [3]:
#!python setup.py build_ext --inplace

In [4]:
from Pricing_Branching import PP_node,PP_node_fairness, get_misclassification, get_split_values,get_misclassification_bag,get_misclassification_bag_sensitive

## Imports

In [5]:
cd /Users/elifrana/Desktop/OCT/n-depth

[Errno 2] No such file or directory: '/Users/elifrana/Desktop/OCT/n-depth'
/Users/elranay/Desktop/FilesAll


In [6]:
## imports
import json
import logging
import pandas as pd
# Define this directive to disable the deprecated NumPy API
#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION
import numpy as np
import copy
from gurobipy import *
import time
from copy import deepcopy
from sklearn import preprocessing as pre
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import concurrent.futures



In [7]:
def print_iteration_separator():
    separator_length = 50
    separator_char = '*'

    separator_line = separator_char * separator_length
    print(separator_line)

In [8]:
def get_shuffle_indices(n):
    arr=np.arange(0,n)
    np.random.shuffle(arr)
    return arr

def split_into_batches(data_x,data_y, n):
    global indices
    global batch_sizes
    batch_size = len(data) // n
    remainder = len(data) % n

    start_idx = 0
    batch_sizes=[]
    for i in range(n):
        batch_end = start_idx + batch_size + (1 if i < remainder else 0)
        batch_sizes.append(batch_end-start_idx)
        yield data_x[indices[start_idx:batch_end]],data_y[indices[start_idx:batch_end]]
        start_idx = batch_end

indices=get_shuffle_indices(2000)
indices

array([1423, 1784, 1671, ...,  543, 1211, 1597])

## Read Data

In [9]:

relpath="./Datasets/"+dataset+"/fold="+str(fold)+"_train.csv"
data = pd.read_csv(relpath)#pd.read_csv("./Datasets/fold=1_train.csv")


X = data.iloc[:,1:]
y = data.iloc[:,:1]



P = [i for i in range(data.shape[1]-1)]
K = list(data.y.unique())
k=len(K)

min_bucket_leaf= 0 # minimum number of data that should fall under each leaf
D = list(range(d))
N = list(range(X.shape[0]))
n = len(N)
n_leaves = 2**d
p = X.shape[1]

y_k = np.zeros((X.shape[0], len(K)))
for k in range(len(K)):
    y_k[np.where(y==K[k])[0],k] = 1

y_k=y_k.astype(np.int32)
x = np.array(X,dtype=np.int32)
y_arr=np.array(y,dtype=np.int32).flatten()-1

## Parameter Generators

In [10]:

#### transfer these to cython!    
def get_alpha(pattern, j, level, split_values=None):
    try:
        if pattern[level] == j and split_values[level] == 1:
            out = 1
        else:
            out = 0
    except:
        print("Incorrect pattern")
        print([pattern, j, level, split_values])
        out = 0

    return out

def get_beta(pattern, j, level, split_values):
    try:
        if pattern[level] == j and split_values[level] == 0:
            out = 1
        else:
            out = 0
    except:
        print([pattern, j, level, split_values])
        print("Incorrect pattern")
        out = 0

    return out


#### transfer these to cython!    
def get_alpha_beta(pattern, j, feature_split_condition):
    """
    This function assumes that the pattern provided is a tuple of lists, with 0-split features being the second entry,
    and 1-split features being the first entry. If the feature split condition matches the position of the feature,
    the function returns one. 
    """
    try:
        if j in pattern[1-feature_split_condition]:
            out = pattern[1-feature_split_condition].count(j)
        else:
            out = 0
    except:
        print("Incorrect pattern")
        print([pattern, j, feature_split_condition])
        out = 0

    return out





## return the solution of the tree:
### WRITE CODE HERE###
## column generator function
def get_column(leaf,d,pattern,p):
    """ 
    Gives the column associated with a certain pattern. The column vector will have length 2^d + (2^(d+1))*p
    Establishes the leaf's connection with all cousins, by order of closeness of cousin degrees.
    The connections at one level are formed for each cousin pair by iterating each feature. This procedure is continued for each level in which the
    pair continues to be associated with each other.

    """
    global sensitive_features
    my_column=np.zeros(2**d)
    my_column[leaf]=1
    
    cons_list=[my_column]
    start=leaf*2*p
    leaf_cons=np.zeros(2**(d+1)*p)
    for feature in range(p):
        for split_condition in range(2): # binary split tree
            leaf_cons[start+2*feature+split_condition]=-1*get_alpha_beta(pattern,feature,1-split_condition)
    cons_list.append(leaf_cons)
    fairness_cons=get_fairness_cons(leaf,pattern,sensitive_features) # 2* len(sensitive features)
    cons_list.append(fairness_cons)
    column=np.concatenate(cons_list)

    return(column.reshape(len(column),1)) # appropriate column shape

def get_w_columns(d,p):
    """
    Takes tree depth and number of features p to return the w columns always present in the model.
    The returned array can be joined horizontally with the pattern constraints.
    Cython version exists, but is not in use.

    """
    global sensitive_features
    number_of_nodes= 2**(d)-1
    len_w = number_of_nodes*p
    len_cons=p*2**(d+1) ## binary splits
    assignment_cons=np.zeros((2**d,len_w))
    all_cons=[assignment_cons]
    

    for leaf in range(2**d):
        

        split_values=1-get_split_values(leaf,d).astype(int)
        w_indices_for_leaf=[0]

        ### obtain the leaf's association with split nodes
        for d_temp in range(1,d):
            values=np.arange(2**d_temp -1,2**d_temp -1 + 2**d_temp)
            w_index=leaf//(2**(d-d_temp))
            w_indices_for_leaf.append(values[w_index])


        ### build the constraint set:
        for feature_ in range(p):
            feature_cons=[]
            for node in range(number_of_nodes):
                cons_temp=np.zeros((2,p))
                if node in w_indices_for_leaf:
                        index=w_indices_for_leaf.index(node)
                        cons_temp[split_values[index],feature_]=1
                feature_cons.append(cons_temp)

            cons_leaf_feature=np.hstack(feature_cons)
            all_cons.append(cons_leaf_feature)
            
    ### add fairness cons
    all_cons.append(np.zeros((2*len(sensitive_features),len_w)))
    all_cons=np.vstack(all_cons)
    
    return all_cons   
        
def get_fairness_cons(leaf,pattern,sensitive_features):
    global n
    global x

    # returns the fairness constraint column for the pair, sensitive features are given as a list
    sigma_list=[]
    for feature in sensitive_features:
        #n_group_sensitive=len(x[np.where(x[:,feature]==1)])
        #n_group_nonsensitive=n-n_group_sensitive
        info_sensitive, info_non_sensitive=get_misclassification_bag_sensitive(x, y_k, pattern, leaf,[feature])
        #sigma= info_sensitive[0]/info_sensitive[1] if info_sensitive[1]> 0 else 0   - info_non_sensitive[0]/info_non_sensitive[1] if info_non_sensitive[1]>0 else 0
        #sigma= (info_sensitive[0] - info_non_sensitive[0])/n#(info_sensitive[1]+info_non_sensitive[1]) if info_sensitive[1]+info_non_sensitive[1]> 0 else 0 
        #sigma= (info_sensitive[0]*n_group_nonsensitive - info_non_sensitive[0]*n_group_sensitive)/(n_group_sensitive*n_group_nonsensitive)
        sigma=(info_sensitive[0] - info_non_sensitive[0])/n
        sigma_list.extend([sigma,-1*sigma])
    column= np.array(sigma_list)
    return column

def get_w_indices(leaf):
    """
    Cython version exists, but is not in use.
    """
    # returns effected indices of each leaf
    w_indices_for_leaf=[0]
    ### obtain the leaf's association with split nodes
    for d_temp in range(1,d):
        values=np.arange(2**d_temp -1,2**d_temp -1 + 2**d_temp)
        w_index=leaf//(2**(d-d_temp))
        w_indices_for_leaf.append(values[w_index])

    return w_indices_for_leaf

def get_affected_leaves(node,w_indices_for_leaves):
    """
    Cython version exists, but is not in use.
    """
    affected_leaves=[]
    ### obtain the leaf's association with split nodes

    for leaf in range(len(w_indices_for_leaves)):
        values=w_indices_for_leaves[leaf]
        if node in values:
            affected_leaves.append(leaf) 

    return affected_leaves

def get_cousins(depth, leaf):
    """
    returns a list of cousins ordered by their degree
    """
    cousins=[[] for level in range(depth)]
    cousins[depth-1]=[i for i in range(2**depth) if i!=leaf]
    for i in range(depth-1,0,-1):
        middle_split= leaf//2**(i)
        cousins[i-1]=cousins[i][-(2**(i)-1):] if middle_split else cousins[i][:2**(i)-1] 


    return cousins



def calculate_miss(x_tilde,y_tilde,j,RHS):
    indices= x_tilde[:,j]==RHS
    x_tilde_2= x_tilde[indices,:]
    y_tilde_2= y_tilde[indices,:]
    n_j= len(x_tilde_2)
    count=np.sum(y_tilde_2,axis=0)
    return n_j-np.max(count)

def calculate_gini(x_tilde,y_tilde,j,RHS):
    indices= x_tilde[:,j]==RHS
    x_tilde_2= x_tilde[indices,:]
    y_tilde_2= y_tilde[indices,:]
    n_j= len(x_tilde_2) # number of data points
    count=np.sum(y_tilde_2,axis=0) #number of ones and zeros
    gini= 1-np.sum(np.square((count/n_j))) #1- sum(p^2) -- sum over classes
    return gini

from more_itertools import distinct_permutations
from itertools import product,combinations,combinations_with_replacement



def find_all_bags(feature_num,leaf,d):
    """
    takes number of features and returns all possible bag combinations
    includes repeats
    """
    split_values=get_split_values(leaf,d)
    max_num=int(np.sum(split_values))
    input_list=np.arange(feature_num)
    input_set = set(input_list)
    result_tuples = []

    
    for i in combinations_with_replacement(input_list, max_num):
        list1 = list(i) # 1 features
        set_temp1=set(list1)
        set2= input_list#list(input_set-set(i)) # 0 features  #### either enable or disable selecting a feature as both 0 and 1 # input_set
        for j in combinations_with_replacement(set2, d-max_num):
            list2 = list(j)  # 0 features
            set_temp2=set(list2)
            
            result_tuples.append((list1, list2))
    
    return result_tuples

def find_all_bags_fixed_feature(feature_num,leaf,d,fixed_feature_info):
    """
    takes number of features and returns all possible bag combinations
    includes repeats
    uses the fixed feature (given in a list of tuples)
    """
    # we have to not use the features above!
    
    split_values=get_split_values(leaf,d)
    #dont_use=set(fixed_feature_info[fixed_feature_info[:,-1]==0][:,1].tolist()) # get non-used features for splits
    fixed_all=fixed_feature_info[fixed_feature_info[:,-1]==1].tolist() # get the fixed features

    input_list=np.arange(feature_num)
    input_set = set(input_list)#s-dont_use
    
    result_tuples = []
    
    for condition in set(get_split_values(leaf,d).astype(int)):

        fixed_pattern_template=([],[])
        counters=[0,0]
    

        for fixed in fixed_all:
            feature=fixed[0]
            #w_index=fixed[0]
            #depth_affected=w_indices.index(w_index)
            #condition= get_split_values(leaf,d).astype(int)[depth_affected]
            fixed_pattern_template[1-condition].append(feature)
            counters[1-condition]+=1
            max_num=int(np.sum(split_values))-counters[0]

            # for every possible split use case!
            for i in combinations_with_replacement(input_set, max_num):
                list1 = list(i) # 1 features
                set_temp1=set(list1)
                set2= input_set#list(input_set-set(i)) # 0 features  #### either enable or disable selecting a feature as both 0 and 1 # input_set
                for j in combinations_with_replacement(set2, d-max_num-sum(counters)):
                    list2 = list(j)  # 0 features
                    set_temp2=set(list2)
                    result_tuples.append((list1+fixed_pattern_template[0], list2+fixed_pattern_template[1])) # add the fixed feature
    
    return result_tuples

def find_all_repeating_bags(feature_num,leaf,d):
    """
    takes number of features and returns all possible bag combinations
    includes repeats
    """
    split_values=get_split_values(leaf,d)
    max_num=int(np.sum(split_values))
    input_list=np.arange(feature_num)
    input_set = set(input_list)
    result_tuples = []

    
    for i in combinations_with_replacement(input_list, max_num):
        list1 = list(i) # 1 features
        set2 = input_set #list(input_set-set(i)) # 0 features
        set_temp1=set(list1)
    
            

        for j in combinations_with_replacement(set2, d-max_num):
            list2 = list(j)  # 0 features
            set_temp2=set(list2)

            if (len(set_temp1)!=len(list1) or len(set_temp2)!=len(list2)) or len(set_temp1.intersection(set_temp2)) >0: #and len(set_temp1.intersection(set_temp2)) ==0: ### for prohibiting invalid patterns #
                ## repeating check and valid check
                result_tuples.append((list1, list2))



    
    return result_tuples

#Cythonized version exists
"""
def get_miss_bag(pattern_bag,leaf,x,y_k):
    # pattern_bag includes a tuple of lists
    global d
    splits=1-get_split_values(leaf,d).astype(int)

    pattern=[] #[0] 

    counter=np.zeros(2,dtype=np.int32)
    #[[0, 0], []]
    for split_condition in splits:
        try:
            pattern.append(pattern_bag[split_condition][counter[split_condition]])
            counter[split_condition]+=1
        except: # a feature is repeated for branching
            pattern.append(pattern_bag[split_condition][counter[split_condition]-1])
        
  
    return get_misclassification(x, y_k, np.array(pattern,dtype=np.int32), leaf)
"""
# Define a custom sorting key
def custom_sort(item):
    return (len(item[0]), len(item[1]))


def get_shuffle_indices(n):
    arr=np.arange(0,n)
    np.random.shuffle(arr)
    return arr

def split_into_batches(data_x,data_y, n):
    global indices
    global batch_sizes
    batch_size = len(data) // n
    remainder = len(data) % n

    start_idx = 0
    batch_sizes=[]
    for i in range(n):
        batch_end = start_idx + batch_size + (1 if i < remainder else 0)
        batch_sizes.append(batch_end-start_idx)
        yield data_x[indices[start_idx:batch_end]],data_y[indices[start_idx:batch_end]]
        start_idx = batch_end
        

## Branching for Pricing Problems Using Beam Search

## Initial Patterns List

empty patterns should be detected

list of all patterns that are valid

In [11]:
#### generate all tuples and randomly add to the initial set
###### NEW PATTERN DEFINITION ####
np.random.seed(42)# set seed (also restart seed!)

### add one level splits!
#### store the set of all patterns
# Sort the couples list using the custom key
all_patterns_list=[find_all_bags(p,leaf,d) for leaf in range(2**d)]
all_patterns_list_sorted=[sorted(all_patterns_list[leaf], key=custom_sort) for leaf in range(2**d)]
all_repeating_patterns_list=[find_all_repeating_bags(p,leaf,d) for leaf in range(2**d)] # ensure repeats

_=[np.random.shuffle(all_repeating_patterns_list[leaf])for leaf in range(2**d)]
pattern_list= [all_repeating_patterns_list[leaf][:len(all_patterns_list[leaf])//10**(d-1)] for leaf in range(2**d)] # add a fraction of repeating patterns

_=[np.random.shuffle(all_patterns_list[leaf])for leaf in range(2**d)]
_=[pattern_list[leaf].extend(all_patterns_list[leaf][:len(all_patterns_list[leaf])//10**(d-1)]) for leaf in range(2**d)]


In [12]:
num_patterns_initial=np.sum([len(pattern_list[i]) for i in range(2**d)])
num_patterns_all=np.sum([len(all_patterns_list[i]) for i in range(2**d)])

In [13]:
print('Number of all valid patterns:',num_patterns_all)

Number of all valid patterns: 2054


In [14]:
print("pattern_list:",pattern_list)

pattern_list: [[([8, 8], []), ([16, 16], []), ([0, 0], []), ([24, 24], []), ([11, 11], []), ([9, 9], []), ([13, 13], []), ([1, 1], []), ([23, 23], []), ([5, 5], []), ([2, 2], []), ([12, 12], []), ([15, 15], []), ([3, 3], []), ([4, 4], []), ([21, 21], []), ([17, 17], []), ([22, 22], []), ([18, 18], []), ([25, 25], []), ([20, 20], []), ([7, 7], []), ([10, 10], []), ([14, 14], []), ([19, 19], []), ([6, 6], []), ([16, 23], []), ([1, 14], []), ([21, 25], []), ([10, 13], []), ([6, 20], []), ([9, 10], []), ([7, 18], []), ([4, 7], []), ([3, 16], []), ([1, 5], []), ([2, 23], []), ([7, 14], []), ([5, 25], []), ([2, 15], []), ([8, 18], []), ([2, 6], []), ([15, 24], []), ([14, 25], []), ([20, 20], []), ([3, 10], []), ([3, 5], []), ([5, 17], []), ([10, 22], []), ([4, 23], []), ([10, 15], []), ([17, 19], []), ([3, 7], []), ([5, 6], []), ([0, 9], []), ([14, 16], []), ([4, 11], []), ([12, 14], []), ([2, 9], []), ([4, 19], []), ([2, 22], [])], [([18], [18]), ([10], [10]), ([7], [7]), ([12], [12]), ([24

In [15]:
### instance log
relpath="./Datasets/"+dataset+"/fold="+str(fold)
filename=relpath+'_instance_log_'+str(d)+'.txt'
with open(filename, 'w') as file:
    K = list(data.y.unique())
    texts=[str(n),str(p),str(num_patterns_all),str(len(K))]
    separator = ","
    data = separator.join(texts)
    file.write(data)

All Patterns

## Master Problem

In [16]:
def solve_partial_OCT_IP_matrix(pattern_list, x, y_k, d,p):
    global n
    global epsilon
    global cons_time
    timer_start=time.time()
    ### Baseline model
    ## the W variables coefficients will be filled before column generation
    cons_basis= get_w_columns(d,p)
    
    
    ### get A matrix ###
    columns = [cons_basis]
    _ = [
        [columns.append(get_column(leaf,d, pattern, p)) for leaf in range(len(pattern_list)) for pattern in pattern_list[leaf]]    
    ] # null list to hide extend output

    miss_and_count=[get_misclassification_bag(x, y_k, pattern_bag, leaf) for leaf in np.arange(len(pattern_list),dtype=np.int32) for pattern_bag in pattern_list[leaf]]
    
    ### MISSS function change here
    
    A= np.concatenate(columns,axis=1)


    ### get b ###
    b = np.concatenate([np.ones(len(pattern_list)),np.zeros(len(A)-len(pattern_list)-2*len(sensitive_features)),np.ones(2*len(sensitive_features))*epsilon]) # RHS
    
    ### get c ###

    c=np.array([miss_and_count[i][0]/n for i in range(len(miss_and_count))])
    c=np.concatenate([np.zeros(cons_basis.shape[1]),c]) ### add 0 obj for w
    
    ### build model ###

    mp = Model('MasterProblemLP')
    #mp.setParam('OutputFlag', 0)
    
    mp.setParam('TimeLimit', 10800)

    # add as many variables as the number of patterns
    mp.addMVar(A.shape[1]-len(miss_and_count), vtype=GRB.BINARY,name='w') 
    mp.addMVar(len(miss_and_count), vtype=GRB.BINARY,name='pattern') 
    signs=['=',]*(len(A)-2*len(sensitive_features)) + ['<']*(2*len(sensitive_features)) # fairness signs
    mp.addMConstr(A, None, signs , b) # change here as well

    #sp.setMObjective(None,c_new,0,None,None,None,GRB.MAXIMIZE)
    mp.setMObjective(None, c, 0.0, xQ_L=None, xQ_R=None, xc=None, sense=GRB.MINIMIZE )

    timer_end=time.time()
    cons_time=timer_end-timer_start

    timer_start=time.time()
    mp.optimize()
    timer_end=time.time()
    return mp, A, b, c,timer_end-timer_start

def solve_partial_OCT_matrix(pattern_list, x, y_k, d,p):
    global n
    
    ### Baseline model
    ## the W variables coefficients will be filled before column generation
    cons_basis= get_w_columns(d,p)
    
    
    ### get A matrix ###
    columns = [cons_basis]
    _ = [
        [columns.append(get_column(leaf,d, pattern, p)) for leaf in range(len(pattern_list)) for pattern in pattern_list[leaf]]    
    ] # null list to hide extend output

    miss_and_count=[get_misclassification_bag(x, y_k, pattern_bag, leaf) for leaf in np.arange(len(pattern_list),dtype=np.int32) for pattern_bag in pattern_list[leaf]]
    
    ### MISSS function change here
    
    A= np.concatenate(columns,axis=1)


    ### get b ###
    b = np.concatenate([np.ones(len(pattern_list)),np.zeros(len(A)-len(pattern_list)-2),np.ones(2*len(sensitive_features))*epsilon]) # RHS
    
    ### get c ###

    c=np.array([miss_and_count[i][0]/n for i in range(len(miss_and_count))])
    c=np.concatenate([np.zeros(cons_basis.shape[1]),c]) ### add 0 obj for w
    
    ### build model ###

    mp = Model('MasterProblemLP')
    mp.setParam('OutputFlag', 0)
    
    # add as many variables as the number of patterns
    mp.addMVar(A.shape[1]-len(miss_and_count), vtype=GRB.CONTINUOUS,name='w',lb=0) 
    mp.addMVar(len(miss_and_count), vtype=GRB.CONTINUOUS,name='pattern',lb=0) 

    signs=['=',]*(len(A)-2) + ['<']*2 # fairness signs
    mp.addMConstr(A, None, signs , b)

    #sp.setMObjective(None,c_new,0,None,None,None,GRB.MAXIMIZE)
    mp.setMObjective(None, c, 0.0, xQ_L=None, xQ_R=None, xc=None, sense=GRB.MINIMIZE )
    mp.optimize()

    return mp, A, b, c




def dual_partial_OCT_matrix(A,b,c):
    ### build model ###
    dual_mp=Model('Dual')

    dual_mp.setParam('OutputFlag', 0)
    
    # add as many variables as the number of constraints
    dual_mp.addMVar(A.shape[0], vtype=GRB.CONTINUOUS,ub=GRB.INFINITY,lb=-GRB.INFINITY,name='pattern') 

    dual_mp.addMConstr(A.T, None, '<' , c)

    #sp.setMObjective(None,c_new,0,None,None,None,GRB.MAXIMIZE)
    dual_mp.setMObjective(None, b, 0.0, xQ_L=None, xQ_R=None, xc=None, sense=GRB.MAXIMIZE )
    dual_mp.optimize()

    return dual_mp




In [17]:
#m,A,b,c=solve_partial_OCT_matrix(all_patterns_list, x, y_k, d,p)

In [18]:
#m.objval

In [19]:
#m,A,b,c,_=solve_partial_OCT_IP_matrix(all_patterns_list, x, y_k, d,p)

In [20]:
#m.ObjVal

In [21]:
#m.Runtime

In [22]:
def printIPVars(IP_model):
    # Print non-zero decision variables, HERE BOTH PATTERN NUMBER AND LEAF NUMBER STARTS FROM 0
    pattern_counter=[0,]*(2**(d))
    leaf_pointer_array=IP_model.getA()[:2**(d)].toarray()
    for i, var in enumerate(IP_model.X):
            leaf=np.where((leaf_pointer_array[:,i]==1))[0][0]    
            if var > 0:
                index= pattern_counter[leaf]
                print(f'Leaf {leaf+1} pattern {index+ 1}: {var}')
                print('pattern:',all_patterns_list[leaf][index])
            pattern_counter[leaf]+=1 #update index

In [23]:
#list_unraveled=[]
#_=[list_unraveled.extend(all_patterns_list[l]) for l in range(2**d)]
#indices=np.where(np.array(m.X)[(2**d-1)*p:])[0]
#for i in indices:
#    print(list_unraveled[i])

## CART 

In [24]:
# Function to recursively extract split features
def extract_split_features(d,tree_structure,leaf):
    """
    Node: 0,1,2,...2**d-1
    """
    empty_pattern=tuple([[] for _ in range(2)])
    nodes=get_w_indices(2**d-leaf-1) # we are doing this because CART builder assumes right splits are postive and left are negative
    splits=get_split_values(leaf,d).astype(int)
    tree_features=tree_structure.feature[tree_structure.feature>=0]
    for node in nodes:
        feature_new = tree_features[node] if node<len(tree_features) else -1
        level=0#(node+1)//(2**(d-1))
        flag=node in np.arange(2**level-1,2**(level+1)-1).tolist()
        while not flag:
            level+=1
            flag=node in np.arange(2**level-1,2**(level+1)-1).tolist()
            
        empty_pattern[1-splits[level]].append(feature_new) if feature_new >=0 else empty_pattern[1-splits[level]].append(feature_old)
        feature_old=feature_new # store this split
    
    return empty_pattern


In [25]:
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn.metrics import accuracy_score,confusion_matrix
clf = tree.DecisionTreeClassifier(max_depth=d)
clf = clf.fit(x, y)
prediction = clf.predict(x)
cart_acc=1-accuracy_score(y,prediction)
print('CART Miss:',cart_acc)

CART Miss: 0.23756858710562412


CART Result


[Text(0.6, 0.8333333333333334, 'x[23] <= 0.5\ngini = 0.682\nsamples = 11664\nvalue = [3888.0, 3840.0, 1.0, 3640.0, 295.0]'),
 Text(0.4, 0.5, 'x[8] <= 0.5\ngini = 0.536\nsamples = 7776\nvalue = [0, 3840, 1, 3640, 295]'),
 Text(0.5, 0.6666666666666667, 'True  '),
 Text(0.2, 0.16666666666666666, 'gini = 0.52\nsamples = 6237\nvalue = [0, 3653, 1, 2288, 295]'),
 Text(0.6, 0.16666666666666666, 'gini = 0.213\nsamples = 1539\nvalue = [0, 187, 0, 1352, 0]'),
 Text(0.8, 0.5, 'gini = 0.0\nsamples = 3888\nvalue = [3888, 0, 0, 0, 0]'),
 Text(0.7, 0.6666666666666667, '  False')]

In [26]:
print('CART Result')
tree.plot_tree(clf)

In [27]:
## store incumbent CART patterns
inc_patterns_CART=[[extract_split_features(d,clf.tree_,leaf)] for leaf in range(2**d)]

In [28]:
## add cart patterns to assoc. pattern list
_=[pattern_list[l].append(inc_patterns_CART[l][0]) for l in range(2**d)]
_=[all_patterns_list[l].append(inc_patterns_CART[l][0]) for l in range(2**d)]

In [29]:
from itertools import combinations_with_replacement

## Solve the IP model

In [30]:
print('Solving IP Model')
IP_model=solve_partial_OCT_IP_matrix(all_patterns_list, x, y_k, d,p)

Solving IP Model
Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-26
Set parameter TimeLimit to value 10800
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 24.2.0 24C101)

CPU model: Apple M3 Pro
Thread count: 11 physical cores, 11 logical processors, using up to 11 threads

Optimize a model with 214 rows, 2136 columns and 10131 nonzeros
Model fingerprint: 0x6e35a38a
Variable types: 0 continuous, 2136 integer (2136 binary)
Coefficient statistics:
  Matrix range     [2e-02, 2e+00]
  Objective range  [8e-04, 5e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 53 rows and 4 columns
Presolve time: 0.01s
Presolved: 161 rows, 2132 columns, 6917 nonzeros
Variable types: 0 continuous, 2132 integer (2132 binary)
Found heuristic solution: objective 0.2904664

Root relaxation: objective 0.000000e+00, 30 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective B

In [31]:
def printIPVars(IP_model):
    # Print non-zero decision variables, HERE BOTH PATTERN NUMBER AND LEAF NUMBER STARTS FROM 0
    inc_patterns=[[] for leaf in range(2**d)]
    pattern_counter=[0,]*(2**(d))
    leaf_pointer_array=IP_model[0].getA()[:2**(d)].toarray()
    for i, var in enumerate(IP_model[0].X[(2**d-1)*p:]):
            leaf=np.where((leaf_pointer_array[:,(2**d-1)*p+i]==1))[0][0]    
            if var > 0:
                index= pattern_counter[leaf]
                print(f'Leaf {leaf+1} pattern {index+ 1}: {var}')
                print('pattern:',all_patterns_list[leaf][index])
                inc_patterns[leaf].append(all_patterns_list[leaf][index])
            pattern_counter[leaf]+=1 #update index
    return inc_patterns

def get_ODM(inc_patterns,d,x,y_k):
    global sensitive_features
    n= len(x)
    ODM=0
    for feature in sensitive_features:
        ODM_feat_p=0
        ODM_feat_m=0

        for leaf in range(2**d):
            info_sensitive, info_non_sensitive=get_misclassification_bag_sensitive(x, y_k, inc_patterns[leaf][0], leaf,[feature])
            #sigma= info_sensitive[0]/info_sensitive[1] if info_sensitive[1]> 0 else 0   - info_non_sensitive[0]/info_non_sensitive[1]>0 if info_non_sensitive[1]>0 else 0
            sigma= (info_sensitive[0] - info_non_sensitive[0])/(n)#/(info_sensitive[1]+info_non_sensitive[1]) if info_sensitive[1]+info_non_sensitive[1]> 0 else 0 
            ODM_feat_p+=sigma
            ODM_feat_m-=sigma
        ODM=max(ODM,max(ODM_feat_p,ODM_feat_m))
    return ODM

def get_DCM(inc_patterns,d,x,y_k):
    global sensitive_features
    global K
    DCM=0
    
    for classOfInterest in range(len(K)):
        for feature in sensitive_features:
                DCM_feat_class=0
                for leaf in range(2**d):
                    info_sensitive, info_non_sensitive=get_misclassification_bag_sensitive_class(x, y_k, inc_patterns[leaf][0], leaf,[feature],classOfInterest)
                    #print(info_sensitive)
                    #sigma= info_sensitive[0]/info_sensitive[1] if info_sensitive[1]> 0 else 0   - info_non_sensitive[0]/info_non_sensitive[1]>0 if info_non_sensitive[1]>0 else 0
                    sigma= (info_sensitive[0] - info_non_sensitive[0])/np.sum(y_k[:,classOfInterest])#/(info_sensitive[1]+info_non_sensitive[1]) if info_sensitive[1]+info_non_sensitive[1]> 0 else 0 
                    DCM_feat_class+=abs(sigma)
                    #print(sigma)
                DCM=max(DCM,DCM_feat_class)
        return DCM

In [32]:
inc_patterns_IP=printIPVars(IP_model)

Leaf 1 pattern 134: 1.0
pattern: ([8, 23], [])
Leaf 2 pattern 228: 1.0
pattern: ([8], [23])
Leaf 3 pattern 320: 1.0
pattern: ([23], [8])
Leaf 4 pattern 51: 1.0
pattern: ([], [8, 23])


In [33]:
print('IP Model Objective:', IP_model[0].objval)
print('IP Model Optimization Time:',IP_model[-1])

IP Model Objective: 0.23756858710562415
IP Model Optimization Time: 0.0414738655090332


In [34]:
IP_ODM=get_ODM(inc_patterns_IP,d,x,y_k)
print('IP Model Train ODM:',IP_ODM)

IP Model Train ODM: 0.29183813443072704


# Test Dataset

In [35]:
relpath="./Datasets/"+dataset+"/fold="+str(fold)+"_test.csv"
data = pd.read_csv(relpath)#pd.read_csv("./Datasets/fold=1_train.csv")

X = data.iloc[:,1:]
y = data.iloc[:,:1]

y_k = np.zeros((X.shape[0], len(K)))
for k in range(len(K)):
    y_k[np.where(y==K[k])[0],k] = 1

y_k=y_k.astype(np.int32)
x = np.array(X,dtype=np.int32)

In [36]:
def get_tree_mis(inc_patterns,d,x,y_k):
    miss=0
    for leaf in range(2**d):
        miss+=get_misclassification_bag(x, y_k, inc_patterns[leaf][0], leaf)[0]
    return miss/len(x)


In [37]:
miss_IP=get_tree_mis(inc_patterns_IP,d,x,y_k)
IP_test_ODM=get_ODM(inc_patterns_IP,d,x,y_k)


In [38]:
print('IP Test Accuracy Score is:', 1-miss_IP)

IP Test Accuracy Score is: 0.7723765432098766


In [39]:
#Training Accuracy
#Test Accuracy
#Time
#Opt Gap

In [40]:
print('IP Model Objective:', IP_model[0].objval)
print('IP Model Optimization Time:',IP_model[-1])

IP Model Objective: 0.23756858710562415
IP Model Optimization Time: 0.0414738655090332


In [41]:
res_MIP=np.array([epsilon,cons_time,IP_model[-1],1-IP_model[0].objval,IP_ODM,IP_model[0].MIPGap, 1-miss_IP,IP_test_ODM])
labels=np.array(['Epsilon','Cons_Time','Time','Train Accuracy','Train ODM','Opt Gap','Test Accuracy', 'Test ODM'])

In [42]:
res_final=np.vstack([labels,res_MIP])

In [43]:
res_final

array([['Epsilon', 'Cons_Time', 'Time', 'Train Accuracy', 'Train ODM',
        'Opt Gap', 'Test Accuracy', 'Test ODM'],
       ['1.0', '1.3160450458526611', '0.0414738655090332',
        '0.7624314128943759', '0.29183813443072704', '0.0',
        '0.7723765432098766', '0.3055555555555556']], dtype='<U32')

In [44]:
# save results
relpath="./Datasets/"+dataset+"/fold="+str(fold)
np.savetxt(relpath+'_res_'+str(d)+'_Fairness_IP_Analysis_'+str(sensitive_features)+'_'+str(epsilon)+'.txt',res_final,fmt='%s',delimiter=',')

In [45]:
import sys

orig_stdout = sys.stdout
f = open(relpath+'_DecisionRules_FairnessIP_'+str(d)+'.txt', 'w')
sys.stdout = f

print(inc_patterns_IP)

sys.stdout = orig_stdout
f.close()